## **Задание 1**
Выполнено: Ханенко Даниилом ИУ6-54Б

[Набор данных](../data/places.csv)

```python
# Наименование столбцов
['ID', 'Name', 'global_id', 'IsNetObject', 'OperatingCompany', 'TypeObject', 'AdmArea', 'District', 'Address', 'PublicPhone', 'SeatsCount', 'SocialPrivileges', 'Longitude_WGS84', 'Latitude_WGS84', 'geoData']
```

⚠️ **Замечание**: 1) для парсинга данных используйте модуль `csv`; 2) для вычисления расстояния используйте формулу гаверсинуса.

1. Рассчитайте расстояние от заданной точки (`lat=55.751244`, `lng=37.618423`) до каждого заведения общепита из набора данных. Выведите первые 10.
2. Рассчитайте расстояние между всеми заведениями общепита из набора данных. Выведите первые 10.
3. Выведите топ-10 наиболее близких и наиболее отдаленных заведений.


In [22]:
import csv
from math import radians, cos, sin, asin, sqrt
from pyspark import SparkContext, SparkConf


input_path = "places.csv"
conf = SparkConf().setAppName("HW1").set("spark.driver.host", "127.0.0.1").set("spark.driver.bindAddress", "127.0.0.1")
sc = SparkContext(conf=conf)


In [23]:
def get_haversin_dist(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [float(lon1), float(lat1), float(lon2), float(lat2)])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * asin(sqrt(a)) * 6371

def parse_csv_line(line):
    return next(csv.reader([line]))


In [24]:
TARGET_LAT = 55.751244
TARGET_LNG = 37.618423

raw_rdd = sc.textFile(input_path)
header = raw_rdd.first()

data_rdd = raw_rdd.filter(lambda x: x != header) \
                  .map(parse_csv_line).cache()


1. Рассчитайте расстояние от заданной точки (`lat=55.751244`, `lng=37.618423`) до каждого заведения общепита из набора данных. Выведите первые 10.

In [25]:
target_dist_rdd = data_rdd.map(lambda row: (
    row[1],
    get_haversin_dist(TARGET_LNG, TARGET_LAT, row[12], row[13])
))

for name, dist in target_dist_rdd.take(10):
    print(f"{name}: {round(dist, 5)}")

МУ-МУ: 4.41145
КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА: 4.14157
Дом 12: 2.97678
Чито-Ра: 3.03484
Бар- буфет «Николай»: 2.82873
Флорентини: 2.88858
Beer Gik: 1.56531
Погребок: 1.56531
Пробка Гриль: 1.64999
TEMPO DI PASTA: 1.64474


2. Рассчитайте расстояние между всеми заведениями общепита из набора данных. Выведите первые 10.

In [26]:
light_rdd = data_rdd.map(lambda x: (x[0], x[1], x[12], x[13]))

all_pairs_rdd = light_rdd.cartesian(light_rdd) \
    .filter(lambda x: x[0][0] < x[1][0]) \
    .map(lambda x: (
        f"{x[0][1]} - {x[1][1]}",
        get_haversin_dist(x[0][2], x[0][3], x[1][2], x[1][3])
    ))

for pair, dist in all_pairs_rdd.take(10):
    print(f"{pair}: {round(dist, 5)}")


МУ-МУ - КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА: 0.68307
МУ-МУ - Флорентини: 1.52344
МУ-МУ - Макдоналдс: 2.62944
МУ-МУ - ЭЛЬ ГАУЧО: 2.05959
МУ-МУ - Темпл бар: 0.06241
МУ-МУ - Бейрут: 1.79928
МУ-МУ - Алтаргана: 1.71954
МУ-МУ - Византия: 1.73678
МУ-МУ - Unlock cafe: 3.34521
МУ-МУ - Брусника: 3.18203


3. Выведите топ-10 наиболее близких и наиболее отдаленных заведений.


In [27]:
min_10 = target_dist_rdd.takeOrdered(10, key=lambda x: x[1])
max_10 = target_dist_rdd.takeOrdered(10, key=lambda x: -x[1])

print("Ближайшие:")
for name, dist in min_10:
    print(f"{name}: {round(dist, 5)}")

print("\nСамые отдаленные:")
for name, dist in max_10:
    print(f"{name}: {round(dist, 5)}")

Ближайшие:
Мареа: 1.05598
Стейк Хаус «Бизон»: 1.059
ЛяМур: 1.06532
БИБЛИОТЕКА Shisha Lounge: 1.06604
Му-Му: 1.06604
Unlock cafe: 1.06843
Папа Джонс: 1.06847
Jimmy Poy: 1.06891
Шоколадница: 1.0699
Настоишная: 1.06995

Самые отдаленные:
МНИТИ: 6.42686
Школа 414: 6.29339
Византия: 6.13249
Школа 435: 6.08727
Мята Lounge: 6.0393
Ля Фантази: 5.95456
ШефДонер: 5.95456
Take & Wake: 5.95456
Пекинский Сад: 5.8992
Гриль и Сувлаки: 5.89551


In [28]:
sc.stop()